In [2]:
from pystac.client import Client
from odc.stac import load
from odc.geo.geom import BoundingBox

In [3]:
# DEP STAC catalog
catalog = "https://stac.staging.digitalearthpacific.io"

# SDB collection 
collection = "dep_s2_sdb" 

In [4]:
bbox = BoundingBox(178.888, -17.535, 179.008, -17.393)
bbox.explore()

In [5]:
client = Client.open(catalog)

items = client.search(
    collections=[collection], bbox=bbox, datetime="2024"
).item_collection()

print(f"Found {len(items)} items in the collection {collection}.")

Found 1 items in the collection dep_s2_sdb.


In [6]:
# Load the DEP data
data = load(
    items,
    bbox=bbox,
    bands=["depth"],
    chunks={}
)

data

<xarray.Dataset> Size: 9MB
Dimensions:      (y: 1648, x: 1337, time: 1)
Coordinates:
  * y            (y) float64 13kB -1.954e+06 -1.954e+06 ... -1.97e+06 -1.97e+06
  * x            (x) float64 11kB 3.216e+06 3.216e+06 ... 3.229e+06 3.229e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables:
    depth        (time, y, x) float32 9MB dask.array<chunksize=(1, 1648, 1337), meta=np.ndarray>

In [7]:
import folium
from ipyleaflet import basemaps

centre = (bbox.bottom + bbox.top) / 2, (bbox.left + bbox.right) / 2
map = folium.Map(location=centre, zoom_start=12, tiles=basemaps.Esri.WorldImagery)
data.depth.odc.add_to(map)

map

In [8]:
data.depth.odc.write_cog("depth.tif")

PosixPath('depth.tif')